# Explorando dataset del consumo de agua. 

In [47]:
#todo chido con las versiones
import sys
import pandas as pd
sys.version

'3.7.4 (default, Sep 22 2020, 18:21:06) \n[Clang 11.0.0 (clang-1100.0.33.17)]'

In [48]:
data = pd.read_csv('agua/consumo-agua.csv')
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

### ¿Cuántas variables tenemos? ¿Cuántas observaciones tenemos?

In [49]:
observaciones, variables = data.shape[0], data.shape[1]
print(f"Tenemos {observaciones} observaciones para {variables} variables.")

Tenemos 71102 observaciones para 17 variables.


In [50]:
data.head()

,Geo Point,Geo Shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
0,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",159.72,2019,Gustavo A. Madero,42.566364,468.23,GUSTAVO A. MADERO,7 DE NOVIEMBRE,53.24,631.00,42.066667,3.05,3,3.05,57250,ALTO
1,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,35.936667,107.81,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,115.13,28.782500,7.32,3,7.32,57253,MEDIO
2,"19.4557195871,-99.1135822797","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,24.586000,122.93,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,197.96,32.993333,75.03,3,75.03,57255,POPULAR
3,"19.4596467168,-99.1044693641","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,0.000000,0.00,GUSTAVO A. MADERO,NUEVA TENOCHTITLAN,0.00,253.53,84.510000,84.51,3,253.53,57267,BAJO
4,"19.4741606185,-99.1467497317","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",56.72,2019,Azcapotzalco,67.436250,539.49,AZCAPOTZALCO,PROHOGAR,56.72,839.35,76.304545,121.57,3,243.14,57330,BAJO


### ¿Cuántas observaciones únicas tenemos por variables? 


In [51]:
data.nunique()

Geo Point               22930
Geo Shape               22922
consumo_total_mixto     24339
anio                        1
nomgeo                     17
consumo_prom_dom        52060
consumo_total_dom       47051
alcaldia                   16
colonia                  1340
consumo_prom_mixto      31911
consumo_total           56015
consumo_prom            62214
consumo_prom_no_dom     37440
bimestre                    3
consumo_total_no_dom    27336
gid                     71102
indice_des                  4
dtype: int64

### ¿Cuántas variables numéricas tenemos? 

In [52]:
#podemos ver todos los tipos de variables
data.dtypes

Geo Point                object
Geo Shape                object
consumo_total_mixto     float64
anio                      int64
nomgeo                   object
consumo_prom_dom        float64
consumo_total_dom       float64
alcaldia                 object
colonia                  object
consumo_prom_mixto      float64
consumo_total           float64
consumo_prom            float64
consumo_prom_no_dom     float64
bimestre                  int64
consumo_total_no_dom    float64
gid                       int64
indice_des               object
dtype: object

In [53]:
numeric = [e for e in data.dtypes.values if e == "float64" or e == "int64"]
print(f"Tenemos {len(numeric)} variables numéricas.")

Tenemos 11 variables numéricas.


### ¿Cuántas variables de fecha tenemos? 

Ninguna. Se puede ver en la impresión de dtypes. La variable "anio" indica el año, pero eso no es suficiente para ser una fecha (no califica como date, time o datetime). 

### ¿Cuántas variables categóricas tenemos? 

Las siguientes pueden ser consideradas variables categóricas: nomgeo, alcaldia, colonia, e indice_des. Bimestre fue contada como numérica, pero es cierto que puede ser analizada como categórica, al igual que año (dado que solo hay cifras para 2019). Por lo tanto, tenemos entre 4 y 6 variables categóricas, dependiendo de si tomamos a bimestre y año como categórica o no.  

### ¿Cuántas variables de texto tenemos? 

Aunque nomgeo, alcaldia, colonia e indice_des son texto, no contienen como tal texto a analizar (como palabras u oraciones). Diríamos que son categóricas.

### Genera el profiling de cada variable. 

### ¿Qué conocemos ahora de este set de datos por variable?

Geo point indica la latitud y longitud de la colonia. 
Geo shape es un diccionario con información geográfica. 
Nomgeo y alcaldía son equivalentes (e indican el nombre de la alcaldía). Hay 16 distintos. 

Colonia indica el nombre de la colonia.
Anio indica el año, y solo hay datos para 2019.
Bimestre indica el bimestre del año, y solo hay datos para el bimestre 1, 2 y 3. 

### ¿Cuántas alcaldías tienes? ¿Cuántos nomgeo tienes? ¿Identificas algún error? 

In [54]:
print(len(data.alcaldia.unique())) #alcaldías distintas
print(len(data.nomgeo.unique()))#nomgeo distintos 

16
17


Sí el error está en la variable nomgeo: se registró "Talpan" en vez de "Tlalpan". 

In [55]:
[e for e in data.nomgeo.unique() if e.endswith('pan')]

['Talpan', 'Tlalpan']

### Transgormar el nombre de las columnas a formato estándar: minúsculas, sin espacios en blanco (cambiar por guiones bajos), sin signos de puntuación.  

In [56]:
data.columns = data.columns.str.lower().str.replace(" ", "_")
data.head()

,geo_point,geo_shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
0,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",159.72,2019,Gustavo A. Madero,42.566364,468.23,GUSTAVO A. MADERO,7 DE NOVIEMBRE,53.24,631.00,42.066667,3.05,3,3.05,57250,ALTO
1,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,35.936667,107.81,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,115.13,28.782500,7.32,3,7.32,57253,MEDIO
2,"19.4557195871,-99.1135822797","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,24.586000,122.93,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,197.96,32.993333,75.03,3,75.03,57255,POPULAR
3,"19.4596467168,-99.1044693641","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,0.000000,0.00,GUSTAVO A. MADERO,NUEVA TENOCHTITLAN,0.00,253.53,84.510000,84.51,3,253.53,57267,BAJO
4,"19.4741606185,-99.1467497317","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",56.72,2019,Azcapotzalco,67.436250,539.49,AZCAPOTZALCO,PROHOGAR,56.72,839.35,76.304545,121.57,3,243.14,57330,BAJO


### Agregar la variable latitud y longitud generadas a partir de geopoint. Pasar la variable latitud y longitud a numérica si no la tomó como numérica.

In [41]:
#sin lo de "float" lo toma como "object"
latitud = [float(e.split(",")[0]) for e in data["Geo Point"]] 
longitud = [float(e.split(",")[1]) for e in data["Geo Point"]]
 
data["latitud"] = latitud 
data["longitud"] = longitud

data.dtypes

Geo Point                object
Geo Shape                object
consumo_total_mixto     float64
anio                      int64
nomgeo                   object
consumo_prom_dom        float64
consumo_total_dom       float64
alcaldia                 object
colonia                  object
consumo_prom_mixto      float64
consumo_total           float64
consumo_prom            float64
consumo_prom_no_dom     float64
bimestre                  int64
consumo_total_no_dom    float64
gid                       int64
indice_des               object
latitud                 float64
longitud                float64
dtype: object

### Eliminar la columna geo_poiny y geo_shape. 

In [42]:
data.drop(columns = ["Geo Point", "Geo Shape"], inplace = True)
data.head()

,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des,latitud,longitud
0,159.72,2019,gustavo_a_madero,42.566364,468.23,gustavo_a_madero,7_de_noviembre,53.24,631.00,42.066667,3.05,3,3.05,57250,alto,19.455260,-99.112662
1,0.00,2019,gustavo_a_madero,35.936667,107.81,gustavo_a_madero,7_de_noviembre,0.00,115.13,28.782500,7.32,3,7.32,57253,medio,19.455260,-99.112662
2,0.00,2019,gustavo_a_madero,24.586000,122.93,gustavo_a_madero,7_de_noviembre,0.00,197.96,32.993333,75.03,3,75.03,57255,popular,19.455720,-99.113582
3,0.00,2019,gustavo_a_madero,0.000000,0.00,gustavo_a_madero,nueva_tenochtitlan,0.00,253.53,84.510000,84.51,3,253.53,57267,bajo,19.459647,-99.104469
4,56.72,2019,azcapotzalco,67.436250,539.49,azcapotzalco,prohogar,56.72,839.35,76.304545,121.57,3,243.14,57330,bajo,19.474161,-99.146750


### Cambiar a minúsculas las columnas alcaldía, colonia e indice_des.

In [58]:
col_text = ["alcaldia", "colonia", "indice_des"]
for col in col_text:
    data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_").replace(".", ""))

### Volver a correr el proceso de identificación de variables numéricas, categóricas, de texto y fechas. 

Variables numéricas: consumo_total_mixto, anio (de nuevo, que puede ser contada como categórica), consumo_promo_dom, consumo_total_dom, consumo_prom_mixto, consumo_total, consumo_prom, consumo_prom_no_dom, consumo_total_no_dom, gid. Una vez más, bimestre puede ser o no contada como numérica. 

Variables de fecha: ninguna. 

Variables de texto: latitud y longitud son texto para pandas, aunque en realidad son coordenadas. No hay variables que contengan texto a analizar. 

Variables categóricas: nomgeo, alcaldia, colonia, indices_des y bimestre. 

### Genera el data profiling por variable. 

#### Nota para el equipo: La siguiente función realiza toda la preparación de los datos. 

In [59]:
def clean_data(path):
    """
    Función para realizar todas las transformaciones necesarias requeridas al 
    dataset del consumo de agua. 
    
    Inputs:
    -------
    
    path: str
          El path en tu compu donde está el archivo csv. ("/Users/etc/etc..")
    
    Returns:
    --------
    
    data: pandas dataframe
          Un dataframe en el formato correcto
    
    """
    #1. cargar la info y arreglar nombres de columnas
    data = pd.read_csv(path)
    data.columns = data.columns.str.lower().str.replace(" ", "_")
    
    #2. Transformar alcaldia, colonia e indice_des
    col_text = ["alcaldia", "colonia", "indice_des"]
    for col in col_text:
        data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_").replace(".", ""))
    
    #3. Agregar latitud y longitud en formato numérico
    latitud = [float(e.split(",")[0]) for e in data["Geo Point"]] 
    longitud = [float(e.split(",")[1]) for e in data["Geo Point"]]
    data["latitud"] = latitud 
    data["longitud"] = longitud
    
    #4. Tirar variables irrelevantes
    data.drop(columns = ["Geo Point", "Geo Shape"], inplace = True)
    
    return data

In [ ]:
#por si hacemos análisis después
#data.groupby(["nomgeo"]).agg({"consumo_prom_dom" : "mean"})